In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d abhia1999/chronic-kidney-disease
!unzip \*.zip && rm *.zip

  0% 0.00/5.61k [00:00<?, ?B/s]
100% 5.61k/5.61k [00:00<00:00, 14.0MB/s]
Archive:  chronic-kidney-disease.zip
  inflating: new_model.csv           


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data
data = pd.read_csv('/content/new_model.csv')  # Replace '/path/to/your/dataset.csv' with the actual path

# Step 2: Prepare Data
X = data[['Bp', 'Sg', 'Al', 'Su', 'Rbc', 'Bu', 'Sc', 'Sod', 'Pot', 'Hemo']]
y = data['Class']  # Assuming 'Target' is the target variable name

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Define a kernel function (e.g., Gaussian kernel)
def gaussian_kernel(x1, x2, sigma=1):
    return np.exp(-np.linalg.norm(x1 - x2) ** 2 / (2 * (sigma ** 2)))

# Step 6: Train and Evaluate the KMKNN model
k = 5  # Number of neighbors
sigma = 1  # Kernel bandwidth
y_pred = []
for x_test in X_test_scaled:
    distances = [gaussian_kernel(x_test, x_train, sigma) for x_train in X_train_scaled]
    k_nearest_indices = np.argsort(distances)[-k:]
    class_votes = {}
    for idx in k_nearest_indices:
        label = y_train.iloc[idx]
        class_votes[label] = class_votes.get(label, 0) + 1
    predicted_class = max(class_votes, key=class_votes.get)
    y_pred.append(predicted_class)

# Step 7: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Display evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.975
Precision: 0.975
Recall: 0.975
F1 Score: 0.975
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96        28
           1       0.98      0.98      0.98        52

    accuracy                           0.97        80
   macro avg       0.97      0.97      0.97        80
weighted avg       0.97      0.97      0.97        80

